# RAG with Azure AI search

In [4]:
# Import required libraries
import os
import json
from dotenv import load_dotenv

from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters
)


from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import json

load_dotenv()

True

In [5]:
# Configure environment variables
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
azure_openai_embedding_dimensions = 1536
index_name = "product_data_csv"

credential = AzureKeyCredential(key)

In [6]:
# Configure OpenAI API
aoai_client = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2023-05-15"
)
credential = AzureKeyCredential(key)

In [7]:
from azure.search.documents.models import VectorizedQuery

search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

# Generate Document Embeddings using OpenAI Ada Model
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def calc_embeddings(text):
    # model = "deployment_name"
    embeddings = aoai_client.embeddings.create(input = [text], model=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME).data[0].embedding
    return embeddings

def do_search(query, fields):
    embedding = calc_embeddings(query)
    vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields=fields)
  
    results = search_client.search(  
        search_text=None,  
        vector_queries= [vector_query],
        select=["name", "description"],
    )  

    for result in results:  
        print(f"Score: {result['@search.score']}")  
        print(f"Name: {result['name']}")  
        print(f"Description: {result['description']}\n")  

In [8]:
# Pure Vector Search 1 field
do_search("nonstick grills", "description_embedding")

# We get non stick grills as the top result and cast iron grills as the second and third result


RetryError: RetryError[<Future at 0x28c755fce80 state=finished raised NameError>]

In [ ]:
# Cross field Pure Vector Search - 2 fields
do_search("nonstick grills", "name_embedding, description_embedding")


In [ ]:
# Pure Multi-Vector weighted Search - 2 fields
from azure.search.documents.models import VectorizableTextQuery
query = "nonstick grills"  

vector_query_1 = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="name_embedding", weight=2)
vector_query_2 = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="description_embedding", weight=0.5)

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query_1, vector_query_2],
    select=["name", "description"],
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}")  
    print(f"Name: {result['name']}")  
    print(f"Description: {result['description']}\n")  

#### Perform a Pure Vector Search with a filter
In my case I don't have a category field in my index, so I will put the code sample for reference only   
#### THIS CODE BELOW WILL NOT WORK UNLESS YOU HAVE A COLUMN TO FILTER BY!!!

In [ ]:
# Perform an hybrid search with a filter 
# in my case I don't have a category field in my index, so I will put the code sample for reference only 
# THIS CODE BELOW WILL NOT WORK UNLESS YOU HAVE A COLUMN TO FILTER BY!!!

from azure.search.documents.models import VectorFilterMode

# Pure Vector Search
query = "nonstick grills"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="description_embedding")

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="category eq 'grills'",
    select=["name", "description"],
)
  
for result in results:  
    print(f"Score: {result['@search.score']}")  
    print(f"Name: {result['name']}")  
    print(f"Description: {result['description']}\n") 